In [ ]:
import weave
weave.use_frontend_devmode()

In [ ]:
initial_entity_name = 'shawn'
initial_project_name = 'run_chain_20_200_10000'
initial_run_id = 'r49iiea8'
initial_metrics = ['metric0', 'metric1']

In [ ]:
from weave.panels import panel_board


varbar = panel_board.varbar()

entity_name_val = varbar.add('entity_name_val', initial_entity_name, hidden=True)
entity = weave.ops.entity(entity_name_val)
entity_name = varbar.add('entity_name',
                        weave.panels.Dropdown(entity_name_val,
                                            choices=weave.ops.viewer().entities().name()))

project_name_val = varbar.add('project_name_val', initial_project_name, hidden=True)
project = weave.ops.project(entity_name_val, project_name_val)
project_name = varbar.add(
    "project_name",
    weave.panels.Dropdown(project_name_val, choices=entity.projects().name()),
)

run_ids_val = varbar.add('run_ids_val', [initial_run_id])
runs = weave.ops.project(entity_name_val, project_name_val).runs().filter(lambda r: r.id().is_in(run_ids_val))
run_ids = varbar.add('run_ids',
                    weave.panels.Dropdown(run_ids_val,
                                          choices=project.runs().id()))

# Get the run history (wandb.log table)
run_history = varbar.add('run_history', runs.history3().concat())

# This holds the zoom boundaries, and is two-way synchronized with the plot zooms
zoom_domain_x = varbar.add('zoom_domain_x', None, hidden=True)

# This is the extent of the step field in our data (the minimum and maxium values)
step_domain = varbar.add('step_domain', weave.ops.make_list(
    a=weave.ops.numbers_min(run_history['_step']),
    b=weave.ops.numbers_max(run_history['_step'])), hidden=True)

# The "domain" used for binning, which is the zoom range if its non-null, otherwise
# the step_extent
bin_domain_x = varbar.add('bin_domain_x', zoom_domain_x.coalesce(step_domain), hidden=True)

main = panel_board.main()

def plot_for_metric(metric_name, run_history, zoom_domain_x, bin_domain_x):
    return weave.panels.Plot(
                        run_history,
                        series=[
                            # Use two plot series, one for the avg line, and one for
                            # min/max area. We could reduce duplication here a bit more
                            # by updating the Plot API to allow common features of series
                            # to be factored out.
                            weave.panels.Series(
                                    run_history,
                                select_functions={
                                    'x': lambda row: row["_step"].bin(
                                        weave.ops.numbers_bins_equal(bin_domain_x, 250))["start"],
                                    'label': lambda row: row.run().name(),
                                    'y': lambda row: row[metric_name].avg()
                                },
                                groupby_dims=['x', 'label'],
                                constants=weave.panels.PlotConstants(
                                    mark='line'
                                )
                            ),
                            weave.panels.Series(
                                run_history,
                                select_functions={
                                    'x': lambda row: row["_step"].bin(
                                        weave.ops.numbers_bins_equal(bin_domain_x, 250))["start"],
                                    'label': lambda row: row.run().name(),
                                    'y': lambda row: row[metric_name].min(),
                                    'y2': lambda row: row[metric_name].max(),
                                },
                                groupby_dims=['x', 'label'],
                                constants=weave.panels.PlotConstants(
                                    mark='area'
                                )
                            )
                        ],
                        domain_x=zoom_domain_x,
                    )


# Automatically show one sub panel per metric.
# TODO: not working yet, type explosion
# all_panels = weave.panels.Each(
#     runs[0].history3()[0].keys()
# )
# all_panels.config.panel = plot_for_metric(all_panels.item_var(), run_history, zoom_domain_x, bin_domain_x)
# main.add('metrics', all_panels)

for metric_name in initial_metrics:
    main.add(metric_name, plot_for_metric(metric_name, run_history, zoom_domain_x, bin_domain_x))                

board = weave.panels.Board(vars=varbar, panels=main)

In [ ]:
board